In [1]:
import hopsworks
import pandas as pd
import joblib
import os
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2


In [2]:
key=""
with open("api-key.txt", "r") as f:
    key = f.read().rstrip()
os.environ['HOPSWORKS_PROJECT']="deloitte"
os.environ['HOPSWORKS_HOST']="6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai"
os.environ['HOPSWORKS_API_KEY']=key

In [3]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai:443/p/119
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
version=1
fv = fs.get_feature_view(name="loans", version=version)

In [5]:
fv.init_batch_scoring(training_dataset_version=4)

In [7]:
def retrieve_data(feature_view):
    batch_data = feature_view.get_batch_data()
    batch_data.drop('id',axis = 1, inplace = True)
    batch_data.drop('earliest_cr_line',axis = 1, inplace = True)
    return batch_data

df = retrieve_data(fv)
df

2023-03-06 17:09:47,473 INFO: USE `deloitte_featurestore`
2023-03-06 17:09:47,926 INFO: SELECT `fg0`.`loan_amnt` `loan_amnt`, `fg0`.`term` `term`, `fg0`.`int_rate` `int_rate`, `fg0`.`installment` `installment`, `fg0`.`sub_grade` `sub_grade`, `fg0`.`home_ownership` `home_ownership`, `fg0`.`annual_inc` `annual_inc`, `fg0`.`verification_status` `verification_status`, `fg0`.`purpose` `purpose`, `fg0`.`dti` `dti`, `fg0`.`earliest_cr_line` `earliest_cr_line`, `fg0`.`open_acc` `open_acc`, `fg0`.`pub_rec` `pub_rec`, `fg0`.`revol_bal` `revol_bal`, `fg0`.`revol_util` `revol_util`, `fg0`.`total_acc` `total_acc`, `fg0`.`initial_list_status` `initial_list_status`, `fg0`.`application_type` `application_type`, `fg0`.`mort_acc` `mort_acc`, `fg0`.`pub_rec_bankruptcies` `pub_rec_bankruptcies`, `fg0`.`zip_code` `zip_code`, `fg0`.`earliest_cr_line_year` `earliest_cr_line_year`, `fg0`.`id` `id`
FROM `deloitte_featurestore`.`loans_1` `fg0`


,loan_amnt,term,int_rate,installment,sub_grade,home_ownership,annual_inc,verification_status,purpose,dti,...,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,zip_code,earliest_cr_line_year
0,3125.0,36 months,16.59,110.78,D1,MORTGAGE,45000.0,Verified,debt_consolidation,4.05,...,1,4182.0,35.7,33.0,f,INDIVIDUAL,1.0,1.0,05113,1978
1,1000.0,36 months,17.56,35.94,D1,RENT,27000.0,Source Verified,other,26.18,...,0,15120.0,29.2,53.0,f,INDIVIDUAL,0.0,0.0,30723,1996
2,19250.0,60 months,15.61,464.15,C5,RENT,43000.0,Verified,debt_consolidation,5.41,...,1,4968.0,27.9,18.0,f,INDIVIDUAL,1.0,1.0,22690,1986
3,18150.0,36 months,19.52,670.09,E2,MORTGAGE,100000.0,Not Verified,other,1.13,...,1,1439.0,11.5,37.0,f,INDIVIDUAL,1.0,1.0,22690,1992
4,10200.0,60 months,19.53,267.58,D5,RENT,45000.0,Verified,debt_consolidation,22.13,...,0,9602.0,19.9,22.0,w,INDIVIDUAL,0.0,0.0,70466,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395214,7500.0,36 months,15.22,260.80,C3,RENT,34500.0,Verified,debt_consolidation,3.34,...,0,2286.0,50.8,20.0,f,INDIVIDUAL,0.0,0.0,48052,1999
395215,10625.0,36 months,13.53,360.72,B5,MORTGAGE,43850.0,Source Verified,debt_consolidation,28.00,...,0,17162.0,35.6,27.0,w,INDIVIDUAL,0.0,0.0,30723,1993
395216,1000.0,36 months,13.66,34.02,C3,RENT,58000.0,Not Verified,vacation,33.29,...,0,38196.0,81.3,21.0,f,INDIVIDUAL,0.0,0.0,29597,1993
395217,24000.0,60 months,13.98,558.19,C3,MORTGAGE,72000.0,Source Verified,credit_card,12.72,...,0,34271.0,83.6,17.0,f,INDIVIDUAL,1.0,0.0,48052,1992


In [9]:
def get_model(project):
    mr = project.get_model_registry()
    model = mr.get_model("lending_model", version = 2)
    model_dir = model.download()
    return joblib.load(model_dir + "/lending_model.pkl")

model = get_model(proj)

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
predictions = model.predict(df)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [15]:
np.unique(predictions, return_counts=True)

(array([0, 1]), array([ 36912, 358307]))